<a href="https://colab.research.google.com/github/ilsong/graduate/blob/master/graduate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **System 정보 확인**
---

In [0]:
# 사용중인 object들 메모리 사용량
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:7]

[('font_list', 528),
 ('sys_font', 528),
 ('nanum_font', 344),
 ('dangling_nodes', 136),
 ('get_pagerank', 136),
 ('get_subgraph', 136),
 ('indegree_dict', 136)]

# **Methods**

In [0]:
import re
import pickle
import json

def parseLink(data):
    ret = re.split("#[sS]-?[0-9]\.?", data.split("|")[0].replace("\\", "").replace("\\", ""))[0].strip()
    return ret

# data.json원본 parse하고 동시에 dead link 없앰
# redirect의경우 "#redirect 문서제목" 이런식으로 되어있으
def parseRawData():
    parsedData = {}
    print("Parse Raw Data Start")
    with open("/content/drive/My Drive/graduate/data.json", "r", encoding='utf-8-sig') as datafile:
        for line in datafile:
            temp = json.loads(line)
            parsedData[temp['title']] = list(filter(None, list(map(parseLink, temp['link']))))

    redirect = {}
    redirect_dead = {}
    for node in parsedData:
        # redirect 문서들
        if len(parsedData[node])==1 and '#redirect ' in parsedData[node][0]:
            link = parsedData[node][0].split("#redirect ")[1]
            # 유효 redirect link
            if link in parsedData and link != node:
                redirect[node] = link
                parsedData[node] = {link: 1}
            # 무효 link, 1-depth self-redirect link -> dead redirect link
            else:
                redirect_dead[node] = 1
                parsedData[node] = []
            continue
        
        # 비 redirect 문서들
        ## 문서안에 self참조 link가 있었음
        temp = {}
        for link in parsedData[node]:
            if link in parsedData and link != node:
                if link in temp: temp[link]+=1;
                else: temp[link]=1;
        parsedData[node] = temp

    for node in parsedData:
        weight_sum = 0
        temp = dict(parsedData[node])
        for link in temp:
            # redirect 있으면 본문서 나올때까지 타고타고 들어간 후, link를 변경
            # A->B->(redirect)C면 A->C로 바꿈
            if link in redirect:
                redir_link = link
                while redir_link in redirect:
                    redir_link = redirect[redir_link]
                # n-depth self redirect link의 경우 삭제
                if redir_link==node:
                    del parsedData[node][link]
                    continue

                if redir_link in parsedData[node]:
                    parsedData[node][redir_link] += parsedData[node].pop(link)
                else:
                    parsedData[node][redir_link] = parsedData[node].pop(link)
                weight_sum += parsedData[node][redir_link]
            # dead redirect link의 경우 삭제
            elif link in redirect_dead:
                del parsedData[node][link]
            else:
                weight_sum += parsedData[node][link]

        # 문서내 유효한 link가 없는경우
        if weight_sum == 0:
            continue
        # 문서내 유효한 link가 있는경우, normalize
        for link in parsedData[node]:
            parsedData[node][link] /= weight_sum

    # Delete Redirect -> 모든 redirect link를 본문서로 옮김
    for redirect_link in redirect:
        del parsedData[redirect_link]
    # Dead Redirect Node -> dead는 삭제해야 함
    for redirect_link in redirect_dead:
        del parsedData[redirect_link]
    del redirect
    del redirect_dead

    print("Parse Raw Data Done")
    return parsedData


# Dict를 파일로 저장(pickle) size크기대로 잘라서 분할 저장함
# size=0 이면 하나의 파일로 저장
def saveDict(filename, _dict, size=100000):
    if not isinstance(filename, str):
        raise ValueError("file name is not string")
    elif not isinstance(_dict, dict):
        raise ValueError("dict is not type of dict")
    elif not isinstance(size, int):
        raise ValueError("size is not type of int")
    elif size < 0:
        raise ValueError("size must be more then 0")

    print("Save Parsed Data Start...")
    # size가 0보다 크면 size만큼 분할해서 저장하고 0이면 한번에 저장
    if size > 0:
        splited_dict = splitDict(_dict, size)
        for idx, e in enumerate(splited_dict):
            pickle.dump(e, open('/content/drive/My Drive/graduate/'+filename + str(idx) + ".bin", "wb"))
    elif size == 0:
        pickle.dump(_dict, open('/content/drive/My Drive/graduate/'+filename + ".bin", "wb"))

    print("Save Parsed Data Done...")


# "[filename][idx].bin" 파일(pickle)을 읽어서 메모리에 로드
# [idx]는 range(total)안에서 반복
def loadDict(filename, index):
    if not isinstance(filename, str):
        raise ValueError("file name is not string")
    elif not isinstance(index, list):
        raise ValueError("index is not list")

    print("Load Parsed Data Start...")
    data = {}
    if index:
        for idx in index:
            data.update(pickle.load(open('/content/drive/My Drive/graduate/'+filename + str(idx) + ".bin", "rb")))
    else:
        data.update(pickle.load(open('/content/drive/My Drive/graduate/'+filename + ".bin", "rb")))
    print("Load Parsed Data Done...")
    return data


# 엄청 큰 Dict를 size크기로 쪼개서 배열에 담음
def splitDict(_dict, size=100000):
    ret = []
    temp = {}
    for idx, key in enumerate(_dict.keys()):
        temp[key] = _dict[key]
        if idx > 0 and (idx % size) == 0:
            ret.append(temp)
            temp = {}
    ret.append(temp)
    return ret


# 노드을 참조하는 부모 노드들을 dict로 리턴
# param: 전체 graph, data.keys()
def indegree_dict(graph, nodes):
    indegree_nodes = {key: [] for key in nodes}
    for node in nodes:
        for dst in graph[node]:
          if dst != node:
            try:
              indegree_nodes[dst].append(node)
              #print("No Key Error: (node, dst): ",node,dst)
            except KeyError:
              #print("Key Error: (dst, node): (",dst,", ",node,")") #," data[node]: ",data[node]," data[dst]: ",data[dst])
              continue
              
    return indegree_nodes

  
# Dangling nodes를 리턴. key: 노드, value: weight
# param: 전체 graph, data.keys()
def dangling_nodes(graph, nodes):
    dangling_nodes_dict = {}
    for node in nodes:
        if len(graph[node]) == 0:
            dangling_nodes_dict[node] = 1
    return dangling_nodes_dict
  
  
# pagerank 리턴
# param: 전체 graph, data.keys()
def get_pagerank(graph, nodes):
    dangling = dangling_nodes(graph,nodes)
    return nx.pagerank(graph, dangling)
  

# 상위 노드, 본인, 하위 노드 및 edge로 구성된 subgraph를 리턴
# pagerank를 이용하여 상/하위 노드 각각 점수 상위 n개씩 그래프에 포함
# param: 전체 graph, 현재 node, 전체 indegree dict, 전체 pagerank, 개수 n
def get_subgraph(graph, cur_node, pagerank, n):
    subgraph = {}

    predgraph_nodes = []
    # cur_node를 가리키고 있는 노드들 pagerank순 n개 구하기
    pred = graph.in_edges(cur_node, data=True)
    pr_local = {}
    for u,v,w in pred:
        pr_local[u] = pagerank[u]
    predgraph_nodes += sorted(pr_local.items(), key=lambda x:x[1], reverse=True)[:n]
    predgraph_nodes = [item[0] for item in predgraph_nodes]
    # 자신포함해야함
    predgraph_nodes.append(cur_node)
    subgraph['predgraph'] = graph.subgraph(predgraph_nodes)
    
    succgraph_nodes = []
    # cur_node가 가리키고 있는 노드들 pagerank순 n개 구하기
    pr_local.clear()
    succ = graph.out_edges(cur_node, data=True)
    for u,v,w in succ:
        pr_local[v] = pagerank[v]
    succgraph_nodes += sorted(pr_local.items(), key=lambda x:x[1], reverse=True)[:n]
    succgraph_nodes = [item[0] for item in succgraph_nodes]
    # 자신포함해야함
    succgraph_nodes.append(cur_node)
    subgraph['succgraph'] = graph.subgraph(succgraph_nodes)

    return subgraph

# TODO: subgraph를 시각화 - 한글 폰트 문제 해결해야함 + cur_node는 표시를 다르게 + weight에 따라 width 조절

# **Data 파일 핸들링**
---

> ## 드라이브 Mount

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


> ## Parse
> data.json 파싱하기 - parseRawData

In [25]:
# data.json 파싱
parsedData = parseRawData()
print("total number of documents: ", len(parsedData))
print("성균관대학교", parsedData["성균관대학교"])

Parse Raw Data Start
Parse Raw Data Done
total number of documents:  445748
성균관대학교 {'성균관': 0.07142857142857142, '성균관대학교/역사': 0.07142857142857142, '성균관대학교/학부': 0.07142857142857142, '성균관대학교/대학원': 0.07142857142857142, '성균관대학교/캠퍼스': 0.07142857142857142, '성균관대학교/기숙사': 0.07142857142857142, '성균관대학교/총학생회': 0.07142857142857142, '성균관대학교/학생문화': 0.07142857142857142, '성균관대학교/동아리': 0.07142857142857142, '성균관대학교/학내언론': 0.07142857142857142, '성균관대학교/사건사고': 0.07142857142857142, '성균관대학교/출신 인물': 0.07142857142857142, '성대사랑': 0.07142857142857142, '삼성의료원': 0.07142857142857142}


> ## Save
> pickle이용해서 data파일로 저장하기 - saveDict

In [4]:
# data파일로 저장하기
saveDict('parsedData', parsedData)
del parsedData

Save Parsed Data Start...
Save Parsed Data Done...


> ## Load
> 1. data를 pickle 파일에서 읽어오기 - loadDict

In [6]:
# data파일에서 읽어오기
data = loadDict("parsedData", [0,1,2,3,4])

Load Parsed Data Start...
Load Parsed Data Done...


# **그래프 모듈**
---

> ## 그래프 생성

In [0]:
import networkx as nx

In [0]:
# 그래프 생성
nx_digraph = nx.DiGraph()

# 노드 추가
nx_digraph.add_nodes_from(data.keys())

# weighted edges 추가
bunch_of_edges = []
for node in data:
    bunch_of_edges += [(node, dst, data[node][dst]) for dst in data[node]]
nx_digraph.add_weighted_edges_from(bunch_of_edges)
del bunch_of_edges, data

print("total graph info - ", "number of nodes: ", nx_digraph.number_of_nodes(), "number of edges: ", nx_digraph.number_of_edges())

> ## 그래프 모델 Save

In [0]:
nx.write_gpickle(nx_digraph, path="/content/drive/My Drive/graduate/graph.bin")

> ## 그래프 모델 Load

In [5]:
nx_digraph = nx.read_gpickle(path="/content/drive/My Drive/graduate/graph.bin")
print("total graph info - ", "number of nodes: ", nx_digraph.number_of_nodes(), "number of edges: ", nx_digraph.number_of_edges())

total graph info -  number of nodes:  445761 number of edges:  14898963


> ## 그래프 분석




> ### PageRank

In [0]:
# pagerank로 분석
pagerank = nx.pagerank(nx_digraph)

In [0]:
# pagerank 결과 save
pickle.dump(pagerank, open("/content/drive/My Drive/graduate/pagerank.bin", "wb"), protocol=4)
del pagerank

In [0]:
# pagerank 결과 load
pagerank = pickle.load(open("/content/drive/My Drive/graduate/pagerank.bin", "rb"))

In [0]:
# 특정 노드와 연결된 노드들중에 pagerank가 높은순 기준으로 subgraph 추출
target_node = "성균관대학교"
subgraph = get_subgraph(nx_digraph, target_node, pagerank, 6)

In [24]:
# target node를 가리키는 노드들로 구성된 그래프 노드들
print("predgraph nodes", subgraph['predgraph'].nodes())

# target node가 가리키는 노드들로 구성된 그래프 노드들
print("succgraph nodes", subgraph['succgraph'].nodes())

predgraph nodes ['라틴어', '일본', '독일어', '성균관대학교', '2018년', '러시아어', '1998년']
succgraph nodes ['삼성의료원', '성균관대학교/동아리', '성균관', '성균관대학교', '성균관대학교/학내언론', '성대사랑', '성균관대학교/캠퍼스']
